# Applications dico


In [51]:
from IPython.display import display, clear_output, HTML, Markdown
from requests_html import HTMLSession
from bs4 import BeautifulSoup
session = HTMLSession()
import re
from pylab import specgram, show
from moviepy.editor import AudioFileClip
from IPython import display as audioD
import IPython.display as ipd
from IPython.display import display, clear_output, HTML, Markdown 
import IPython.display as ipd
import ipywidgets as widgets
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None 

## 1-  Réaliser un scrapper de fichier audio 


#### 1.1. Obtenir l'url du mp3 en anglais britannique et anglais américain d'un mot vedette cible


Objectif : récupérer les audios et traductions

In [52]:
def scrapping(df):
    path="https://dictionary.cambridge.org/fr/dictionnaire/anglais/"
    listeUK = []
    listeUS = []
    
    for mot in df["motVedette"] :
        lien=path+mot
        response = session.get(lien)
        status = response.status_code
        html_doc = response.content
        
        #on récupère le body de notre html
        
        soup = BeautifulSoup(response.text,'html.parser')
        souplettesBody = soup.find('body')
        
        # on filtre sur les balises de type audio
        
        soupletteAside = souplettesBody.findAll('amp-audio')
        soupletteAsideuk = soupletteAside[0].find('source')
        soupletteAsideus = soupletteAside[1].find('source')

        rechercher = soup.findAll("div", {"class" : "pos-header dpos-h"})

        mp3uk = soupletteAsideuk['src']
        mp3us = soupletteAsideus['src']

        base_url='https://dictionary.cambridge.org'
        audiouk = base_url+mp3uk
        audious = base_url+mp3us
        
        #audio us et uk 
        
        listeUK.append(audiouk)
        listeUS.append(audious)
                                  
    df['lienUK']=pd.Series(listeUK).values
    df['lienUS']=pd.Series(listeUS).values
    return df

In [53]:
def Audio(df):
    clear_output()
    listeWid = widgets.Output()
    lU = widgets.Output()
    sU = widgets.Output()

    for numLigne in range(len(df)):
        
        labelUK = widgets.Label("UK")
        itemsUK = audioD.Audio(url = df["lienUK"][numLigne])
        spectroUK = df["lienUK"][numLigne]
    
        labelUS = widgets.Label("US")
        itemsUS = audioD.Audio(url = df["lienUS"][numLigne])
        audioUK = widgets.Output()
        audioUK.append_display_data(itemsUK)
        audioUS = widgets.Output()
        audioUS.append_display_data(itemsUS)
        spectroUS = df["lienUS"][numLigne]
        
        lU = widgets.HBox([labelUK,audioUK,labelUS,audioUS])

        # Les mots
        labelMot = widgets.Label(df["motVedette"][numLigne])
        labelPron = widgets.Label(str(df["pron"][numLigne]))
        label = widgets.HBox([labelMot, labelPron])


        listeWid.append_display_data(widgets.VBox([label,lU,sU]))
                        
    return listeWid

Objectif : Produire par programmation un enrichissement du dictionnaire de Buchanan 1757

In [55]:
cheminDico = r"C:\Users\Ryme2\OneDrive\Documents\CNAM_3A\Machine Learning\[NAUGE]_Youb Ryme\cnam-athon-pronunciation-main\data\dictionary\1757_Buchanan-J.csv" 
dfDicoBuch = pd.read_csv(cheminDico, delimiter=";", encoding="utf8")

dfDicoBuch.sample(n=5)


,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
10833,2,prŏʹxy,NaN,NaN,NaN,n.,16.0,172
13456,1,swăp,NaN,NaN,NaN,v.,57.0,208
7549,2,_k_nāʹv_e_ry,NaN,NaN,NaN,n.,27.0,126
13054,2,ſteeʹrage,NaN,NaN,NaN,n.,15.0,203
3713,3,dĭſcĕʹrnment,NaN,NaN,NaN,n.,22.0,71


## 2- Produire par programmation un enrichissement du dictionnaire de Buchanan 1757

#### 2.1 -  une colonne motVedette en utilisant le mapping du fichier Buchanan_PronChar_counts.csv

In [59]:
cheminMap = r"C:\Users\Ryme2\OneDrive\Documents\CNAM_3A\Machine Learning\[NAUGE]_Youb Ryme\cnam-athon-pronunciation-main\data\dictionary\Buchanan_PronChar_counts.csv"
dfMap = pd.read_csv(cheminMap, delimiter=";", encoding="utf8")

dfMap.sample(n=5)

,char,occ,graph
4,c,5500,c
48,у,3,y
6,s,2935,s
56,а,2,a
54,),28,NaN


In [60]:
dfMap = dfMap.replace(np.nan,'', regex = True)

In [61]:
def correctionTexte(ancienMot,dfMap):
    Resultat = ""
    
    for lettre in ancienMot: 
        dfRowFinded = dfMap[dfMap['char'] == lettre]
        
        if len(dfRowFinded == 1):
            Resultat += dfRowFinded.iloc[0]['graph']
            
        else:
            Resultat += '?'
            
    return(Resultat)

In [62]:
dfDicoBuch['motVedette'] = dfDicoBuch.apply(lambda x: correctionTexte(x['pron'], dfMap),1)

#### 2.2 - Création des colonnes Sampa et Ipa

In [63]:
CheminBigRichSample=r"C:\Users\Ryme2\OneDrive\Documents\CNAM_3A\Machine Learning\[NAUGE]_Youb Ryme\cnam-athon-pronunciation-main\data\dataframe\BigiRichSample.csv"
dfBRS = pd.read_csv(CheminBigRichSample, delimiter=";", encoding="utf8")

In [64]:
DfDico = dfDicoBuch.merge(dfBRS, how='inner',left_on= 'motVedette', right_on='hwd')
del DfDico['hwd']
DfDico.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,Sampa,ipa
2898,2,ĕʹdging,NaN,NaN,NaN,n.,28.0,78,edging,E dZ i N,ɛ?iŋ
1954,2,coоpʹ_e_r,NaN,NaN,NaN,n.,22.0,59,cooper,k u p 3:r,kupɜ:r
1566,1,clŏth,NaN,NaN,NaN,n.,8.0,52,cloth,k l O: T,klOːθ
8685,1,ſtĭck,NaN,NaN,NaN,n.,35.0,203,stick,s t I k,stɪk
7585,2,rĕlieʹf,NaN,NaN,NaN,n.,49.0,180,relief,r\ i l i: f,ɹiliːf


#### 2.3 sauvegarder la nouvelle matrice obtenue dans ./../data/dataframe/BuchananRich.csv

In [65]:
DfDico.to_csv(r"C:\Users\Ryme2\OneDrive\Documents\CNAM_3A\Machine Learning\[NAUGE]_Youb Ryme\cnam-athon-pronunciation-main\data\dataframe\BuchananRich.csv")

In [66]:
cheminBigi = r"C:\Users\Ryme2\OneDrive\Documents\CNAM_3A\Machine Learning\[NAUGE]_Youb Ryme\cnam-athon-pronunciation-main\data\dataframe\BuchananRich.csv"
Bigi = pd.read_csv(cheminBigi, delimiter=",", encoding="utf8")
Bigi.head(3)

,Unnamed: 0,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,Sampa,ipa
0,0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus,{ b @ k @ s,æbəkəs
1,1,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon,@ b { n d @ n,əbændən
2,2,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned,@ b { n d @ n d,əbændənd


In [67]:
dfBigi=Bigi[["motVedette","nbSyll","pron","Sampa","ipa"]]
dfBigi.head(3)

,motVedette,nbSyll,pron,Sampa,ipa
0,abacus,3,aʹbăcus,{ b @ k @ s,æbəkəs
1,abandon,3,ābăʹndon,@ b { n d @ n,əbændən
2,abandoned,4,ābăʹndŏned,@ b { n d @ n d,əbændənd


### 3 - Construire une UI permettant de filtrer les entrées du dictionnaire enrichi de Buchanan

In [68]:
filtreHWD = widgets.Text(value="",description='filtre hwd',layout={'width': '250px'} )
filtreIPA = widgets.Text(value="",description='filtre ipa',layout={'width': '250px'} )
occMax = widgets.IntSlider(min=0,max=10,step=1,description='Occ.Max:',value=1)
bouton = widgets.Button(description='Démarrer', icon = "check")

In [69]:
def on_button_clicked(_):
    nbLigne = occMax.value
    HWD = filtreHWD.value 
    IPA = filtreIPA.value
    df = ""

    with dataFrame:
        clear_output()
        if (HWD != ""):
            df = dfBigi[(dfBigi.motVedette.str.match(HWD))]
            if(IPA != ""):
                df = df[(df.pron.str.match(IPA))]
        elif (IPA != ""):
            df = dfBigi[(dfBigi.pron.str.match(IPA))]

        if(len(df) > nbLigne):
            df = df.sample(n=nbLigne).copy()
        df = df.reset_index()    
        display(df)
        
    with audio :
        clear_output()
        df = scrapping(df)  
        
        listeAudio = Audio(df) 
        display(listeAudio)
        
            
bouton.on_click(on_button_clicked)

In [70]:
dataFrame = widgets.Output()
audio = widgets.Output()

In [72]:
bouton1 = widgets.HBox([filtreHWD, filtreIPA, occMax, bouton])
bouton2 = widgets.HBox([dataFrame])
bouton3 = widgets.HBox([audio])

interface = widgets.VBox([bouton1,bouton2,bouton3])
interface